In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv
/kaggle/input/model-results/check-point/config.json
/kaggle/input/model-results/check-point/trainer_state.json
/kaggle/input/model-results/check-point/training_args.bin
/kaggle/input/model-results/check-point/tokenizer.json
/kaggle/input/model-results/check-point/tokenizer_config.json
/kaggle/input/model-results/check-point/scheduler.pt
/kaggle/input/model-results/check-point/model.safetensors
/kaggle/input/model-results/check-point/special_tokens_map.json
/kaggle/input/model-results/check-point/optimizer.pt
/kaggle/input/model-results/check-point/vocab.txt
/kaggle/input/model-results/check-point/rng_state.pth


In [2]:
!pip install peft

  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/8b/1b/aee2a330d050c493642d59ba6af51f3910cb138ea48ede228c84c204a5af/peft-0.7.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.4 MB/s eta 0:00:00


In [3]:
!pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [4]:
from transformers import(
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model 
import evaluate
import torch
import numpy as np
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
data = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv")

In [6]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

dataset = Dataset.from_pandas(data)

# Use train_test_split to split the dataset into training and validation sets
train_data, validation_data = train_test_split(data, test_size=0.2, shuffle=True)

train_dataset= Dataset.from_pandas(train_data)
validation_dataset= Dataset.from_pandas(validation_data)

# Create a DatasetDict with keys "train" and "validation"
dataset = DatasetDict({"train": train_dataset, "validation": validation_dataset})

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven', '__index_level_0__'],
        num_rows: 35894
    })
    validation: Dataset({
        features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven', '__index_level_0__'],
        num_rows: 8974
    })
})

In [8]:
model_checkpoint = 'distilbert-base-uncased'

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=2)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def tokenize(examples):
    text = examples["text"]
    
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        max_length=512,
        truncation=True,
    )
    
    return tokenized_inputs

In [11]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    
tokenized_dataset = dataset.map(tokenize,batched=True)

  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 35894
    })
    validation: Dataset({
        features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 8974
    })
})

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    predictions,labels = p
    predictions = np.argmax(predictions,axis=1)
    return {"accuracy":accuracy_score(predictions,labels)}    

In [15]:
pert_config = LoraConfig(
    task_type="SEQ_CLS",
    r=4,
    lora_alpha=32,
    lora_dropout=0.01,
    target_modules = ['q_lin']
)

In [16]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
model = get_peft_model(model,pert_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


In [17]:
#hyperparameters
lr = 1e-3
batch_size=4
num_epochs = 8

tags = TrainingArguments(
    output_dir = model_checkpoint +'-lora-text-classification',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs = num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to = "none"
)

In [18]:
trainer = Trainer(
    model=model,
    args=tags,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = compute_metrics
)

In [19]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.138800,0.109536,0.981056
2,0.194800,0.176093,0.975708
3,0.254700,0.292190,0.962447
4,0.224600,0.239237,0.965233
5,0.237600,0.233509,0.970582
6,0.174600,0.228027,0.973256
7,0.141900,0.129091,0.979719
8,0.122000,0.133591,0.982394


TrainOutput(global_step=71792, training_loss=0.19316505660504893, metrics={'train_runtime': 7016.3069, 'train_samples_per_second': 40.926, 'train_steps_per_second': 10.232, 'total_flos': 3.729865787537155e+16, 'train_loss': 0.19316505660504893, 'epoch': 8.0})

In [20]:
trainer.save_model('./model')

In [21]:
tokenizer.save_pretrained("./tokenizer")

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

## Test Inference

In [22]:
# test = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")

# test_dataset = Dataset.from_pandas(test)

# ttd = test_dataset.map(tokenize,batched=True)

In [23]:
# tp = trainer.predict(ttd)

In [24]:
# tp

In [25]:
# logits = tp.predictions

# probs = (np.exp(logits)/np.sum(np.exp(logits),axis=-1,keepdims=True))[:,0]

# sub = pd.DataFrame()

# sub['id'] = test['id']
# sub['generated'] = probs

# sub.to_csv('submission.csv',index=False)

# sub.head()

In [26]:
# model.to('cpu')

In [27]:
# test = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")

# from scipy.special import softmax
# test_predictions = []
# for text in test["text"]:
#     inputs = tokenizer.encode(text,return_tensors="pt").to('cpu')
    
#     logits = model(inputs).logits
#     prediction = softmax(logits,axis=-1)[:,1]
#     test_predictions.append(prediction)

In [28]:
# test["generated"] = test_predictions
# test[["id", "generated"]].to_csv("submission.csv", index=False)